# 1.3 Introduction to Information Retrieval

Here we work with a data set scraped from eBay.  The data contains 9895 item titles and descriptions.

First we load the data file and _normalise_ the text - removing certain characters and converting to lower case.  

In [1]:

import pandas as pd
import math

from scipy.sparse import csr_matrix

from bokeh.plotting import figure, output_notebook, show, vplot
from bokeh.charts import Bar, Scatter, BoxPlot
from bokeh.charts.attributes import CatAttr
from bokeh.models import ColumnDataSource

from sys import getsizeof

In [2]:
import csv
import re

with open("data/bike-items.txt") as f:
    r = csv.reader(f, delimiter=',', quotechar='"')
    rgx = re.compile(r'\b[a-zA-Z]+\b') 
    docs = [ (' '.join(re.findall(rgx, x[0])).lower(), ' '.join(re.findall(rgx, x[1])).lower())  \
                for i,x in enumerate(r) if i > 1 ]
    
print(docs[0][0],docs[0][1])

items_t = [ d[0] for d in docs ] # item titles
items_d = [ d[1] for d in docs ] # item descriptions
items_i = range(0, len(items_t)) # item id


('cycling bicycle mtb bike fixie gloss carbon fiber riser bar handlebar', 'description feature easy to use made of high quality carbon fiber with the special design can save for a long time the carbon fiber handlebar is made of high quality carbon fiber so that you can use it relieved this quick disassembling carbon fiber handlebar is easy to use and one of the best gifts to your friends specification material carbon fiber color black handlebar clamp diameter mm length package included x cycling carbon fiber rise')


## Exercise Set 1 - Term Frequency

Let's start with the first 10 item titles from our corpus:

In [3]:
corpus = items_t[0:5]
print(corpus)

['cycling bicycle mtb bike fixie gloss carbon fiber riser bar handlebar', 'bicycle rims x red speed internal hub wheel set beach cruiser bike', 'mavic crossride mountain bike wheels and wtb weirwolf tires', 'new kcnc arrow alloy stem black', 'rotor qxl aero oval road chainring']


We will start by computing the frequency of terms in the *entire* corpus.  We will do this by enumerating over the corpus of documents, tokenizing the documents and count the frequency of tokens.   The easiest way is to build a python dictionary where the key is the token and the value is the count.  You can review python dictionaries in the [docs](https://docs.python.org/2/tutorial/datastructures.html).

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Here is a part completed code snippet to compute term frequency.  
Complete this code to correctly populate the term frequency dictionary.

In [4]:
tf = {}
for doc in corpus:
    for word in doc.split():
        # << COMPUTE ERM FREQUENCY DICTIONARY >> CODE HERE
        ## HIDE
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1
        ## HIDE

print(tf)

{'and': 1, 'set': 1, 'bicycle': 2, 'cruiser': 1, 'tires': 1, 'fixie': 1, 'oval': 1, 'speed': 1, 'internal': 1, 'mountain': 1, 'cycling': 1, 'handlebar': 1, 'gloss': 1, 'chainring': 1, 'bike': 3, 'black': 1, 'new': 1, 'beach': 1, 'red': 1, 'kcnc': 1, 'wheel': 1, 'rotor': 1, 'fiber': 1, 'hub': 1, 'rims': 1, 'mavic': 1, 'aero': 1, 'stem': 1, 'alloy': 1, 'wtb': 1, 'carbon': 1, 'riser': 1, 'bar': 1, 'qxl': 1, 'crossride': 1, 'arrow': 1, 'weirwolf': 1, 'mtb': 1, 'x': 1, 'wheels': 1, 'road': 1}


We can simplify by using a [Counter](https://docs.python.org/2/library/collections.html#collections.Counter) rather than a dictionary.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'> 
Take a look at the docs for the `Counter` collection.  
Complete this function definition to compute term frequency using the `Counter`.

In [5]:
from collections import Counter

def get_tf(corpus):
    tf = Counter()
    for doc in corpus:
        for word in doc.split():
            # << CODE HERE
            ## HIDE
            tf[word] += 1
    return tf

tf = get_tf(corpus)
print(tf)

Counter({'bike': 3, 'bicycle': 2, 'and': 1, 'set': 1, 'cruiser': 1, 'tires': 1, 'fixie': 1, 'oval': 1, 'speed': 1, 'internal': 1, 'mountain': 1, 'cycling': 1, 'handlebar': 1, 'gloss': 1, 'chainring': 1, 'black': 1, 'new': 1, 'beach': 1, 'red': 1, 'kcnc': 1, 'wheel': 1, 'rotor': 1, 'fiber': 1, 'hub': 1, 'rims': 1, 'mavic': 1, 'aero': 1, 'stem': 1, 'alloy': 1, 'wtb': 1, 'carbon': 1, 'riser': 1, 'bar': 1, 'qxl': 1, 'crossride': 1, 'arrow': 1, 'weirwolf': 1, 'mtb': 1, 'x': 1, 'wheels': 1, 'road': 1})


The Counter does not give us a real speed advantage - since it does more work.   For these tiny data sets we do not see any difference - however in Python 3 it is faster than a default dictionary.  Often times best way to test performance is to time code execution.

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'> 
We should get used to thinking about performance.   
We can use the Jupyter Notebook [magics](http://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb) to time the execution.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Run the code to compute the term frequency for the full corpus of item titles.  
What is the frequency of the terms 'unicycle', 'bicycle' and 'tricycle'?

In [6]:
tf = get_tf(items_t)

# Print tf for 'unicycle'
## HIDE
print(tf['unicycle'])
print(tf['bicycle'])
print(tf['tricycle'])


5
3544
13


The term frequency can also be computed for each document - the term frequency is a crude measure of the "aboutness" of a document.  For short documents, such as eBay item titles, terms do not occur very frequently.  In longer documents the term frequency is a form of compression and summarization.

We can store the document term frequency in a dictionary, where the key is the document id and the value is the a nested dictionary of document terms and their counts.

For example consider the corpus of three documents:

1. 'mountain bike red'
2. 'road bike carbon'
3. 'bike helmet'

The document term frequencies would be:

| id | document term frequencies |
|----|------------------|
| 1  | { 'mountain' : 1, 'bike' : 1, 'red' : 1 } |
| 2  | { 'road' : 1, 'bike' : 1, 'carbon' : 1 } |
| 3  | { 'bike' : 1, 'helmet' : 1 } |


<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Now compute **document term frequencies** for the full corpus of item titles.  
Print out the document term frequencies for 3 randomly selected documents - what was the highest frequency term for each?

In [7]:
def get_tfd(corpus):
    tfd = {}
    for i,doc in enumerate(corpus):
        tfd[i]={}
        # << DOCUMENT TERM FREQUENCY >> CODE HERE
        ## HIDE
        for word in doc.split():
            if word in tfd[i]:
                tfd[i][word] += 1
            else:
                tfd[i][word] = 1
    return tfd
            
    
tfd = get_tfd(items_t)
tfd[234]

{'black': 1,
 'blue': 1,
 'giro': 1,
 'ii': 1,
 'milky': 1,
 'nib': 1,
 's': 1,
 'sante': 1,
 'shoes': 1,
 'white': 1,
 'women': 1}

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Repeat this time computing the **document term frequencies** for the full corpus of item descriptions.  
Print out the document term frequency for 3 randomly selected documents - what was the highest frequency term for each?

In [8]:
# << COMPUTE TFD FOR ITEM DESCRIPTIONS >> CODE HERE
## HIDE
tfd = get_tfd(items_d)
tfd[234]

{'accurately': 2,
 'also': 1,
 'and': 1,
 'answer': 1,
 'answered': 1,
 'are': 1,
 'as': 4,
 'ask': 2,
 'at': 1,
 'best': 1,
 'black': 1,
 'blue': 1,
 'but': 1,
 'buying': 1,
 'can': 1,
 'clothing': 1,
 'describe': 1,
 'descriptions': 1,
 'experience': 1,
 'feel': 2,
 'for': 1,
 'free': 2,
 'giro': 1,
 'have': 4,
 'if': 3,
 'ii': 1,
 'in': 2,
 'interested': 1,
 'it': 1,
 'items': 1,
 'jae': 1,
 'make': 1,
 'may': 1,
 'milky': 1,
 'more': 1,
 'nib': 1,
 'not': 1,
 'of': 1,
 'one': 1,
 'our': 3,
 'please': 1,
 'positive': 1,
 'possible': 1,
 'promptly': 1,
 'questions': 2,
 's': 1,
 'sale': 1,
 'sales': 1,
 'sante': 1,
 'shoes': 1,
 'specific': 1,
 'than': 1,
 'them': 1,
 'these': 1,
 'to': 4,
 'try': 2,
 'we': 6,
 'which': 1,
 'white': 1,
 'will': 1,
 'women': 1,
 'you': 3,
 'your': 1}

## Exercise Set 2 - Term Frequency Ranking and Boolean Matching

Whilst the document term frequency dictionary in the previous section `tfd` is a compact way to store the term frequency it is not efficient for analysis.  A term frequency matrix is a more effective way to store the data.  

For example consider the corpus of three documents:

1. 'mountain bike red'
2. 'road bike carbon'
3. 'bike helmet'

There is a toal vocabulary of six terms [ 'mountain', 'bike' , 'red', 'road', 'carbon', 'helmet' ].

Each document count be represented as a 3 x 6 element vectors:

| Document ID | mountain | bike | red | road | carbon | helmet |
|-------------|----------|------|-----|------|--------|--------|
| 1 - 'mountain bike red' | 1 | 1 | 1 | 0 | 0 | 0 |
| 2 - 'road bike carbon' | 0 | 1 | 0 | 1 | 1 | 0 |
| 3 - 'bike helmet' | 0 | 1 | 0 | 0 | 0 | 1 |

Those arrays can be stacked naturally into a matrix - one row per document, one column per term.  We call this matrix the term frequency matrix.

To compute the term frequency matrix we have to first compute the lexicon (set of terms) in our corpus.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Review the docs for the [set](https://docs.python.org/2/library/stdtypes.html#set-types-set-frozenset) type. Note - sets do not contain duplicates and can be used to dedupe tokens.
Complete the `get_lexicon()` function definition so that it returns a list of unique terms across a given corpus of documents.  Validate with the small test corpus.

In [9]:
def get_lexicon(corpus):
    lexicon = set()
    # << COMPUTE SET OF TERMS IN CORPUS >> CODE HERE
    ## HIDE
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    ## HIDE
    return list(lexicon)
    
test_corpus = ['mountain bike red','road bike carbon','bike helmet']
lexicon = get_lexicon(test_corpus)

Now we have our lexicon we can compute a document term frequency matrix.  We will store our document term frequency vectors in a `list`.  Note we could also store them in a `dictionary` where the key is the document_id.  

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Complete the code snippet below to compute the term frequency vector for each document.  
Store the term frequency vectors in the list `tfm`.  Validate the results with the test corpus.

In [10]:
lexicon = get_lexicon(test_corpus)

tfm =[]
for doc in test_corpus:
    tfv = [0]*len(lexicon)
    for term in doc.split():
        # << COMPUTE DOCUMENT TERM FREQUENCY VECTOR tfv AND APPEND TO tfm >> CODE HERE
        ## HIDE
        tfv[lexicon.index(term)] += 1
        ## HIDE
    tfm.append(tfv)
    
print(tfm)

[[1, 0, 1, 1, 0, 0], [0, 0, 1, 0, 1, 1], [0, 1, 1, 0, 0, 0]]


Since we are going to reuse the tfm let's create a function that takes as argument the corpus and returns the lexicon and the tfm.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Copy your code snippets from the previous two exercises into the function definition below.    
Test your function by computing 'tfm' on the test corpus verifying your results before computing tfm for the item_titles corpus.

In [11]:
def get_tfm(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        # << COMPUTE SET OF TERMS IN CORPUS >> CODE HERE
        ## HIDE
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)
        ## HIDE
        
    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        tfv = [0]*len(lexicon)
        for term in doc.split():
            # << COMPUTE DOCUMENT TERM FREQUENCY VECTOR AND APPEND TO tfm >> CODE HERE
            ## HIDE
            tfv[lexicon.index(term)] += 1
            ## HIDE
        tfm.append(tfv)
        
    return tfm, lexicon


test_corpus = ['mountain bike red','road bike carbon','bike helmet']
tfm, lexicon = get_tfm(test_corpus)


<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'> 
As our corpus increases so does the sparsity of the term frequency matrix - most elements have value zero.  
We can use more efficient [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix) storage to save memory.  More details [here](http://localhost:8888/notebooks/1.3%20Introduction%20to%20Information%20Retrieval.ipynb#Sparse-Term-Frequency-Matrix).  

### Boolean Search

We are now in a position to write our first ranking function.  Now we have the term frequency matrix we can use it to find documents that contain words included in a user specified query.  We will start by simply returning the documents from the corpus that match any terms in the query and rank by the raw frequency of matching terms. 

More specifically our algorithm for 'boolean search' proceeds as follows:

1. Convert query to query vector using the lexicon for the corpus
2. Compute a ranking score for each document by taking the [dot product](https://en.wikipedia.org/wiki/Dot_product) of the query vector and each document's term frequency vector
3. Sort the documents by score

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
The function definion `get_results()` converts the user query `qry` into a vector using the supplied lexicon.
Complete the function by providing the code to compute the score of each document.  Test using a bike related query such as 'led bike light'.  Do you get relevant results?  

HINT : Here is a [gist](https://gist.github.com/mattwg/60910d90a8987e271212) that shows how to compute the dot product between two vectors!

In [12]:
def get_results(qry, tfm, lexicon):
    qrv = [0]*len(lexicon)
    for term in qry.split():
        if term in lexicon:
            qrv[lexicon.index(term)] = 1

    results = []      
    for i, tfv in enumerate(tfm):
        score = 0
        # << COMPUTE DOCUMENT SCORE >> CODE HERE
        ## HIDE
        score = sum([ xy[0] * xy[1] for xy in zip(qrv,tfv)])
        ## HIDE
        results.append([score, i])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results


def print_results(results,n, head=True):
    ''' Helper function to print results
    '''
    if head:    
        print('\nTop %d from recall set of %d items ordered by tf-idf:' % (n,len(results)))
        for r in results[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    else:
        print('\nBottom %d from recall set of %d items ordered by tf-idf:' % (n,len(results)))
        for r in results[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    

tfm, lexicon = get_tfm(items_t)
results = get_results('led bike light', tfm , lexicon)
print_results(results,10)



Top 10 from recall set of 9893 items ordered by tf-idf:
	6.00 - frog waterproof bike light set led white front light led red rear light
	4.00 - cycling bike bicycle led front light head light torch mount aaa
	4.00 - waterproof usb rechargeable led bike light set bright headlight free light
	4.00 - niterider tl sl led bike tail light red rear flashing bike safety
	4.00 - sets bright bike bicycle waterproof led head light led rearlight us seller
	4.00 - lm cree led cycling front bike bicycle light headlight only light
	4.00 - led tire valve stem caps neon light bike bicycle car auto wheel light
	4.00 - ultra bright waterproof silicon led bicycle light set led front rear light
	4.00 - usb cycling xml led front bike light bicycle light headlamp headlight
	4.00 - portable usb rechargeable bicycle bike tail rear light led safety warning light


### Inverted Index

This search across documents is expensive - especially if the score for many documents is zero!  To solve this problem we can create an inverted index.  An inverted index can be used to filter out documents that do not contain any of the keywords in the query before computing the ranking score.  

Using our example mini-corpus:

1. 'mountain bike red'
2. 'road bike carbon'
3. 'bike helmet'

There is a toal vocabulary of six terms [ 'mountain', 'bike' , 'red', 'road', 'carbon', 'helmet' ].  An inverted index will map each of these terms to the document in which the document can be found.  

| key | value |
|-----|-------|
| 'mountain' | [ 1 ] |  
| 'bike' | [1, 2, 3] |  
| 'red' | [1] |  
| 'road' |  [2] | 
| 'carbon' | [2] | 
|  'helmet' | [3] |

We could store an inverted index in a dictionary where the key is the term and the value is the document id.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
We will create an inverted index as a python dictionary keyed on the token.  
Complete the code snippet below to create the inverted index.  Validate with the test corpus. 

In [19]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        # << POPULATE INVERTED INDEX >> CODE HERE
        ## HIDE
        for word in doc.split():
            if word in idx:
                idx[word].append(i)
            else:
                idx[word] = [i]
        ## HIDE
    return idx

test_corpus = ['mountain bike red','road bike carbon','bike helmet']
idx = create_inverted_index(test_corpus)

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Now we can create an inverted index for all the item titles.  We can use the set intersection method to find all the documents that match the query 'led bike light'.  Run the code below checking the titles of some of the results that match query terms.

In [21]:
idx = create_inverted_index(items_t)
print(set(idx['led']).intersection(set(idx['bike'])).intersection(set(idx['light'])))
print(items_t[2061])

set([9559, 2061, 2062, 8212, 31, 8229, 4134, 8238, 4143, 8248, 8250, 4159, 8258, 8261, 2118, 80, 4181, 8278, 91, 4193, 6244, 6247, 9237, 6282, 2195, 2196, 6297, 166, 4274, 2227, 6175, 2239, 4294, 4296, 6345, 4299, 4300, 8406, 8408, 6361, 2267, 2272, 8426, 4331, 4335, 6386, 6388, 245, 6407, 6417, 6419, 6424, 8474, 6434, 2339, 8485, 4398, 306, 8499, 4404, 4406, 316, 4413, 8515, 2372, 6469, 2375, 2387, 6497, 2402, 8553, 6514, 2451, 374, 8571, 384, 6533, 8583, 8590, 6544, 6547, 4501, 2459, 4512, 423, 8623, 4532, 2498, 8652, 4557, 8655, 472, 8665, 8670, 2529, 6629, 8681, 4592, 6230, 6662, 6664, 2569, 8715, 2589, 6686, 6235, 6694, 6697, 4658, 8756, 8761, 8763, 6733, 594, 8790, 599, 4699, 8798, 8807, 618, 8811, 4722, 6772, 2682, 2697, 2706, 4758, 4762, 4769, 4770, 2734, 6839, 2749, 8896, 2753, 2754, 8901, 6859, 717, 4817, 6867, 4821, 731, 6880, 6883, 8934, 4839, 4841, 9684, 8954, 4865, 773, 6918, 6920, 8970, 8972, 6926, 8978, 6934, 796, 8992, 4897, 2850, 9692, 6965, 6971, 4924, 829, 6974, 290

We can now improve on our first ranking function.  This time only scoring the 

In [ ]:

def get_results2(qry, idx):

    score = Counter()
    terms = qry.split()
    for term in terms:
        for doc in idx[term]:
            score[doc] += 1
            
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;


idx = create_inverted_index(items_t)
%timeit results = get_results2('front rear back led bike light', idx)

print_results(results,10)


We get a lot of documents in the recall set since many match on one of the words - bike is present in almost every other document!

In [ ]:
df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

output_notebook(hide_banner=True)
p = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
show(p)


## Inverse Document Frequency (IDF)

It would seem sensible to down weight words that are very common in the corpus - the word 'bike' in a query is not as discriminating as the word 'front'. IDF is a way to quantify how common or rare a term is in the corpus.  It is computed by taking the log of the inverse fraction of the number of documents in which the term appears divided by the total number of documents.  To avoid division by zero it is common to add 1 to the number of documents in which the term appears.  

IDF is already partially computed when we constructed the inverted index - it is the number of documents the term apears in - in otherwords the length of the document list in the inverted index

In [ ]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                if i in idx[word]:
                    # Update document's frequency
                    idx[word][i] += 1
                else:
                    # Add document
                    idx[word][i] = 1
            else:
                # Add term
                idx[word] = {i:1}
    return idx

def idf(term, idx, n):
    return math.log( float(n) / (1 + len(idx[term])))

idx = create_inverted_index(items_t)


df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()],
                  'idf':[idf(x, idx, len(items_t)) for x in idx.keys()]})

output_notebook(hide_banner=True)
p1 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)

p2 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='idf',
        plot_width=800, plot_height=400)

p = vplot(p1, p2)

show(p)

## Ranking by TF-IDF

We can now combine term frequency and inverse document frequency when computing the score for each item in the recall set.  Until now we have just computed the score as the raw frequency of the query terms in each document.  Now we want to weight the raw frequency by the inverse documetn frequency.

In [ ]:
def get_results3(qry, idx, n):
    score = Counter()
    for term in qry.split():
        if term in idf:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;

In [ ]:
idx = create_inverted_index(items_t)
results = get_results3('front led bike light', idx, len(items_t))

print_results(results,10)

## Problematic queries!

With this corpus we cannot search for mountain bikes without returning a heap of accesories:

In [ ]:
idx = create_inverted_index(items_t)
results = get_results3('mountain bike', idx, len(items_t))

print_results(results,10)

We need to penalise items where there are many more terms in the query.  For example the terms "mountain" and "bike" only make up 2 / 12 terms in the "oakley mens automatic mountain mtb factory lite mountain bmx bike gloves large" yet it scores highly because there is no penalty for all the other terms in the item title.

In addition this scheme create discrete levels based on combination of word frequency:


In [ ]:
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'title':[items_t[x[1]] for x in results]})

d = df.groupby('score').first().reset_index()

r1 = re.compile('(bike)')
r2 = re.compile('(mountain)')

for i, t in enumerate(d.title):
    n1 = r1.findall(t)
    n2 = r2.findall(t)
    print('%d x Bike, %d x Mountain, Score = %0.2f'%(len(n1),len(n2),d.score[i]))
    

In [ ]:
# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook(hide_banner=True)
p = Scatter(df, x='score', y='length')
show(p)

Ideally we do not want scores to be the same for lots of documents.  We could try by boosting the score for documents that are shorter than average.

In [ ]:
def get_results4(qry, corpus):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)
    score = Counter()
    for term in qry.split():
        if term in idf:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i *  ( f / (float(d[doc]) / d_avg) )
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;

In [ ]:
results = get_results4('mountain bike', items_t)
print_results(results, 10)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)

## Okapi BM25

In [ ]:
def get_results5(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)                
    score = Counter()
    for term in qry.split():
        if term in idf:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i * (( f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc]) / d_avg)))))
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;

In [ ]:
results = get_results5('mountain bike', items_t)
print_results(results, 10)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)
    

In [ ]:
## Can we make the above interactive?
